<a href="https://colab.research.google.com/github/Rosie-Brigham/colab-gists/blob/main/model_2c.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is the same as model 4, but with a mere 3 classes!

In [ ]:
# Imports
import torch
import torch.nn as nn # All neural network modules, nn.Linear, nn.Conv2d, BatchNorm, Loss functions
import torch.optim as optim # For all Optimization algorithms, SGD, Adam, etc.
import torchvision.transforms as transforms # Transformations we can perform on our dataset
import torchvision
import os
import pandas as pd
from skimage import io
from torch.utils.data import Dataset, DataLoader # Gives easier dataset managment and creates mini batches
from PIL import Image

In [ ]:
!git clone https://github.com/monumentalconservation/training-set-circle-6.git
import os
os.getcwd()

labels = pd.read_csv(r'training-set-circle-6/image-list.csv')
test_labels = pd.read_csv(r'training-set-circle-6/testing-image-list.csv')

train_path = r'training-set-circle-6/images/'
test_path = r'training-set-circle-6/testing-images/'

Cloning into 'training-set-circle-6'...
remote: Enumerating objects: 68, done.
remote: Counting objects: 100% (68/68), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 176 (delta 11), reused 66 (delta 10), pack-reused 108
Receiving objects: 100% (176/176), 152.16 MiB | 47.33 MiB/s, done.
Resolving deltas: 100% (11/11), done.


In [ ]:
labels.head()


,id,none,some,a lot,substantial,extensive
0,6687,1,0,0,0,0
1,6690,1,0,0,0,0
2,6725,1,0,0,0,0
3,6771,1,0,0,0,0
4,6917,1,0,0,0,0


In [ ]:
def quantify_rainfall(row):
  if row['none'] == 1:
    val = 0
  elif row['some'] == 1:
    val = 1
  elif row['a lot'] == 1:
    val = 1
  elif row['substantial'] == 1:
    val = 2
  elif row['extensive'] == 1:
    val = 2
  else:
    return 
  return val

In [ ]:
# Create nice datasets to pass into dataloader

labels['class'] = labels.apply(quantify_rainfall, axis=1)
labels = labels.drop(['none',	'some',	'a lot',	'substantial',	'extensive'], axis=1)

test_labels['class'] = test_labels.apply(quantify_rainfall, axis=1)
test_labels = test_labels.drop(['none',	'some',	'a lot',	'substantial',	'extensive'], axis=1)


In [ ]:
labels.head()

,id,class
0,6687,0
1,6690,0
2,6725,0
3,6771,0
4,6917,0


Thirs attempt at classification - following this tutorial: https://www.youtube.com/watch?v=ZoZHd0Zm3RY&ab_channel=AladdinPersson

In [ ]:
# Dataloader
class DampDataset(Dataset):
    def __init__(self, dataframe, root_dir, transform=None):
        self.annotations = dataframe
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        img_path = os.path.join(self.root_dir, str(self.annotations.iloc[index, 0]) + '.jpg')
        image = io.imread(img_path)
        y_label = torch.tensor(int(self.annotations.iloc[index, 1]))
        if self.transform:
            image = self.transform(image)

        return (image, y_label)


In [ ]:

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Transforms
damp_transform = transforms.Compose(
    [transforms.ToPILImage(),
     transforms.Resize((256,256)), # needed as tuple because images in dataset are different
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])


# Hyperparameters
num_epochs = 50;
batch_size = 10;
learning_rate = 0.0005;
num_classes = 3

# Load Data
dataset = DampDataset(dataframe = labels, root_dir = 'training-set-circle-6/images/', transform = damp_transform)
test_dataset = DampDataset(dataframe = test_labels, root_dir = 'training-set-circle-6/testing-images/', transform = damp_transform)


In [ ]:

# Create the dataloaders
train_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

# Model
model = torchvision.models.googlenet(pretrained=True)
model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss() 
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


Downloading: "https://download.pytorch.org/models/googlenet-1378be20.pth" to /root/.cache/torch/hub/checkpoints/googlenet-1378be20.pth


In [ ]:
# This inherits from AlexNet not CNN - not sure what the difference is, find that out
class AlexNet(nn.Module):
    def __init__(self, num_classes=3):
        super(AlexNet, self).__init__()
    
        self.features = nn.Sequential(nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
                                      nn.ReLU(inplace=True),
                                      nn.MaxPool2d(kernel_size=3, stride=2),
                                      nn.Conv2d(64, 192, kernel_size=5, padding=2),
                                      nn.ReLU(inplace=True),
                                      nn.MaxPool2d(kernel_size=3, stride=2),
                                      nn.Conv2d(192, 384, kernel_size=3, padding=1),
                                      nn.ReLU(inplace=True),
                                      nn.Conv2d(384, 256, kernel_size=3, padding=1),
                                      nn.ReLU(inplace=True),
                                      nn.Conv2d(256, 256, kernel_size=3, padding=1),
                                      nn.ReLU(inplace=True),
                                      nn.MaxPool2d(kernel_size=3, stride=2),)
    
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        
        self.classifier = nn.Sequential(nn.Dropout(),
                                        nn.Linear(256 * 6 * 6, 4096),
                                        nn.ReLU(inplace=True),
                                        nn.Dropout(),
                                        nn.Linear(4096, 4096),
                                        nn.ReLU(inplace=True),
                                        nn.Linear(4096, num_classes),)
    
    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), 256 * 6 * 6)
        x = self.classifier(x)
        return x


In [ ]:
#instance of the Conv Net

# cnn = AlexNet()
cnn = model

#loss function and optimizer
criterion = nn.CrossEntropyLoss();
optimizer = torch.optim.Adam(cnn.parameters(), lr=learning_rate);

In [ ]:
# Check accuracy on training to see how good our model is
def check_accuracy(loader, model, type):
    num_correct = 0
    num_samples = 0
    model.eval()
    
    with torch.no_grad():
        for x, y in loader:
            x = x.cuda()
            y = y.cuda()
            model.cuda()
            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)
        print(f"{type}")
        print(f'Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}') 
    
    model.train()

In [ ]:
from torch.autograd import Variable


losses = [];
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images, labels = images.cuda(), labels.cuda() #UNCOMMENT when training with googlenet!
        images = images.float()
        # print(images.shape)
        # Forward + Backward + Optimize
        optimizer.zero_grad()
        outputs = cnn(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        losses.append(loss.item());
        
        
        if (i+1) % 10 == 0:
            print ('Epoch : %d/%d, Iter : %d/%d,  Loss: %.4f' 
                   %(epoch+1, num_epochs, i+1, len(dataset)//batch_size, loss.item()))
            check_accuracy(train_loader, cnn, 'train')
            check_accuracy(test_loader, cnn, 'test')

Epoch : 1/50, Iter : 10/12,  Loss: 2.1132
train
Got 103 / 125 with accuracy 82.40
test
Got 16 / 25 with accuracy 64.00
Epoch : 2/50, Iter : 10/12,  Loss: 0.4895
train
Got 99 / 125 with accuracy 79.20
test
Got 15 / 25 with accuracy 60.00
Epoch : 3/50, Iter : 10/12,  Loss: 0.0323
train
Got 119 / 125 with accuracy 95.20
test
Got 17 / 25 with accuracy 68.00
Epoch : 4/50, Iter : 10/12,  Loss: 0.0053
train
Got 113 / 125 with accuracy 90.40
test
Got 12 / 25 with accuracy 48.00
Epoch : 5/50, Iter : 10/12,  Loss: 0.2122
train
Got 120 / 125 with accuracy 96.00
test
Got 18 / 25 with accuracy 72.00
Epoch : 6/50, Iter : 10/12,  Loss: 1.1177
train
Got 124 / 125 with accuracy 99.20
test
Got 15 / 25 with accuracy 60.00
Epoch : 7/50, Iter : 10/12,  Loss: 0.0706
train
Got 125 / 125 with accuracy 100.00
test
Got 19 / 25 with accuracy 76.00
Epoch : 8/50, Iter : 10/12,  Loss: 0.1392
train
Got 125 / 125 with accuracy 100.00
test
Got 18 / 25 with accuracy 72.00
Epoch : 9/50, Iter : 10/12,  Loss: 0.0136
train

In [ ]:
# # Now we save the model, so we can play with this after lunch
# from google.colab import drive
# drive.mount('/content/drive')

# # #Saving model to drive
# model_save_name = 'googlenet-70e.pt'
# path = F"/content/drive/MyDrive/PhD/models/model_five/googlenet-30e-b10-lr0005.pt" 
# torch.save(model.state_dict(), path)

Mounted at /content/drive


In [ ]:
# LOAD PREVIOUSLY TRAINED MODEL HERE
# path = "/content/drive/MyDrive/PhD/models/model_five/googlenet-50e-10b.pt" 
# model.load_state_dict(torch.load(path))
# model.eval()

In [ ]:
print("Checking accuracy on Training Set")
check_accuracy(train_loader, cnn)

print("Checking accuracy on Test Set")
check_accuracy(test_loader, cnn)

Checking accuracy on Training Set
Got 124 / 125 with accuracy 99.20
Checking accuracy on Test Set
Got 18 / 25 with accuracy 72.00


In [ ]:
# test-the-model
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in train_loader:
        images = images.cuda()
        labels = labels.cuda()
        outputs = cnn(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
          
    print('Training Accuracy of the model: {} %'.format(100 * correct / total))



Training Accuracy of the model: 96.0 %


From here, these snippets are taken from this colab: https://colab.research.google.com/github/pytorch/tutorials/blob/gh-pages/_downloads/17a7c7cb80916fcdf921097825a0f562/cifar10_tutorial.ipynb#scrollTo=TzrMwxulq03d

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# see if this works...

classes = ('none', 'some', 'loads')

dataiter = iter(test_loader)
images, labels = dataiter.next()

def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.figure(figsize = [20,20])
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# print images


imshow(torchvision.utils.make_grid(images, nrow=5))
print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(10)))

print(labels)

In [ ]:
class_correct = list(0. for i in range(3))
class_total = list(0. for i in range(3))
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images = images.cuda()
        labels = labels.cuda()
        outputs = cnn(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(3):
    # print(class_total)
    print('Accuracy of %5s : %2d %%' % (classes[i], float(100) * class_correct[i] / class_total[i]))

NameError: ignored